In [ ]:
%pylab inline
import glob
import pandas as pd
import os
import datetime as dt
import seaborn as sns

import behav
from behav import plotting, utils, loading
data_path = '/mnt/cube/RawData/Zog/'

subjects = (
            'B1109',
            'B1222'
            )

## Loads Data

In [ ]:
%%time
behav_data = behav.loading.load_data_pandas(subjects,data_path)

## plot performance over past two weeks in calendar

In [ ]:
for subj,data in behav_data.iteritems():
    behav.plotting.plot_filtered_performance_calendar(subj,data,num_days=20)    

##plot accuracy per stim

In [ ]:
figsize(16.0, 8.0)
for subj,df in behav_data.items():
    behav.plotting.plot_filtered_accperstim(subj, df, extract_stim_names=True)

## Plot percent correct by block with confidence bounds

In [ ]:
figsize(16.0, 8.0)
for subj,df in behav_data.items():
    behav.plotting.plot_ci_accuracy(subj, df)

##plot accuracy today

In [ ]:
figsize(16.0, 8.0)
for subj,df in behav_data.items():
    behav.plotting.plot_daily_accuracy(subj, df, x_axis='trial_num')

# Plot number of trials and feeds for past week

In [ ]:
behav.plotting.plot_trial_feeds(behav_data)